In [5]:
#pacotes necessarios
#install.packages("e1071") 
#install.packages("caret")
#install.packages("randomForest")
#install.packages("Metrics")
library("caret")
library("randomForest")
library("Metrics")


Anexando pacote: 'Metrics'


Os seguintes objetos são mascarados por 'package:caret':

    precision, recall




In [1]:
# carregamento da base
dados <- read.csv("databases/2 - Volume - Dados.csv")
head(dados)

,PROJETO,CLONE,Idade,Hdom,AreaBasal,Volume
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,Fazenda Velha,C3,21.5,8.8,3.2,12.1
2,Fazenda Velha,A3,21.9,10.2,3.9,66.8
3,Fazenda Velha,A3,26.1,10.1,4.2,72.1
4,Fazenda Velha,A4,22.0,10.0,4.2,35.9
5,Parada,B7,23.4,15.3,4.3,44.5
6,Fazenda Velha,A2,25.9,10.5,4.4,63.9


In [4]:
#separacao da base em treino e teste
set.seed(2034)
indices <- createDataPartition(dados$Volume, p = 0.8, list = FALSE)

treino <- dados[indices, ]
teste <- dados[-indices, ]

In [8]:
#treinando o modelo
set.seed(2034)
rf <- train(Volume ~ ., data = treino, method = "rf")
rf

predict_rf <- predict(rf, teste)
rmse(teste$Volume, predict_rf)
r2 <- function(observado, predito) {
  return(1 - (sum((predito - observado)^2)/sum((observado - mean(observado))^2)))
}


r2(teste$Volume, predict_rf)


Random Forest 

246 samples
  5 predictor

No pre-processing
Resampling: Bootstrapped (25 reps) 
Summary of sample sizes: 246, 246, 246, 246, 246, 246, ... 
Resampling results across tuning parameters:

  mtry  RMSE      Rsquared   MAE     
   2    53.49488  0.9467127  43.24763
   6    43.09511  0.9533489  33.72283
  10    43.15854  0.9522889  32.97885

RMSE was used to select the optimal model using the smallest value.
The final value used for the model was mtry = 6.

[1] 30.94597

[1] 0.974638

In [10]:
#experimento com cross-validation
ctrl <- trainControl(method = "cv", number = 10)

rf <- train(Volume ~ ., data = treino, method = "rf", trControl = ctrl)
rf

predict_rf <- predict(rf, teste)
rmse(teste$Volume, predict_rf)
r2(teste$Volume, predict_rf)

Random Forest 

246 samples
  5 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 221, 222, 221, 222, 221, 222, ... 
Resampling results across tuning parameters:

  mtry  RMSE      Rsquared   MAE     
   2    51.24744  0.9569486  41.65150
   6    38.22169  0.9648168  30.13990
  10    37.15388  0.9660598  28.46621

RMSE was used to select the optimal model using the smallest value.
The final value used for the model was mtry = 10.

[1] 30.48653

[1] 0.9753855

In [11]:
#experimentando outros valores de mtry
tune_grid <- expand.grid(mtry = c(2, 5, 7, 9))

rf <- train(Volume ~ ., data = treino, method = "rf", trControl = ctrl, tuneGrid = tune_grid)
rf

predict_rf <- predict(rf, teste)
rmse(teste$Volume, predict_rf)
r2(teste$Volume, predict_rf)

Random Forest 

246 samples
  5 predictor

No pre-processing
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 222, 221, 222, 222, 220, 221, ... 
Resampling results across tuning parameters:

  mtry  RMSE      Rsquared   MAE     
  2     50.94183  0.9587024  41.83122
  5     37.36790  0.9684745  30.37207
  7     36.07477  0.9699581  29.10867
  9     35.42113  0.9701285  28.05560

RMSE was used to select the optimal model using the smallest value.
The final value used for the model was mtry = 9.

[1] 30.84373

[1] 0.9748053

In [12]:
#predicao de novos casos
dados_novos_casos <- read.csv("databases/2 - Volume - Dados - Novos Casos.csv")
View(dados_novos_casos)

dados_novos_casos$Volume <- NULL

predict_rf <- predict(rf, dados_novos_casos)

resultado <- cbind(dados_novos_casos, predict_rf)
View(resultado)

PROJETO,CLONE,Idade,Hdom,AreaBasal,Volume
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
Parada,C3,21.5,8.8,4.2,?
Santa Maria,A3,22.9,14.1,5.6,?
Jacutinga,A2,59.4,22.3,13.0,?


,PROJETO,CLONE,Idade,Hdom,AreaBasal,predict_rf
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
1,Parada,C3,21.5,8.8,4.2,42.58816
2,Santa Maria,A3,22.9,14.1,5.6,108.59594
3,Jacutinga,A2,59.4,22.3,13.0,288.74715
